# I Construction de la base de données

## Régression classique

In [6]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import ruptures as rpt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import cross_val_score

data_folder = 'datas/'

final_data = pd.read_csv(os.path.join(data_folder, 'final_data.csv'), parse_dates=['DATE'])
selected_columns = ['BOP', 'business_insolvencies', 'firms_creation', 'natality_rate', 'unemployment_rate', 'Valuation', 'Exch_rate', 'Fixed rate', 'negotiable_debts']
df = final_data[selected_columns]
data_for_regression = final_data[selected_columns]

data_for_regression = data_for_regression.dropna()

X = data_for_regression.drop(columns=['negotiable_debts'])
Y = data_for_regression['negotiable_debts']
X = sm.add_constant(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialisation du modèle de régression linéaire avec l'option robuste

model = sm.RLM(Y_train, X_train, M=sm.robust.norms.HuberT()).fit()
print(model.summary())

predictions = model.predict(X_test)

# Évaluation de la performance du modèle
mse = mean_squared_error(Y_test, predictions)
r2 = r2_score(Y_test, predictions)

print(f'\nMean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

matrix = pd.concat([Y, X], axis=1).to_numpy()

                    Robust linear Model Regression Results                    
Dep. Variable:       negotiable_debts   No. Observations:                  137
Model:                            RLM   Df Residuals:                      128
Method:                          IRLS   Df Model:                            8
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Sat, 30 Dec 2023                                         
Time:                        21:21:58                                         
No. Iterations:                     4                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   9.45e+

## Détection de grandes périodes et points de rupture

Nous venons donc de réaliser une régression linéaire classique afin de trouver des corrélations, significatives ou non, entre nos régresseurs et l'objet d'étude (la dette négociable de l'Etat). Cependant, il semble essentiel de souligner que ces corrélations peuvent changer au cours du temps. L'économie n'est pas un champ d'étude atemporel, certains coefficients devant des régresseurs peuvent positifs et significatifs pendant une certaine période, par exemple après la crise de 2008, puis perdre en significativité voire changer de signe pendant une période plus calme. Nous avons donc eu l'idée de trouver des grandes périodes de l'économie entre 2009 et 2023. Ces périodes se trouvent en cherchant des "points de rupture", c'est à dire des instants où les coefficients devant les régresseurs tendent à changer. 

In [7]:
# Détection de changement avec ruptures pour chaque coefficient
breakpoints = []
for i in range(X.shape[1]):
    algo = rpt.Pelt(model="rbf").fit(matrix[:, [0, i + 1]])
    result = algo.predict(pen=10)  # Vous pouvez ajuster le paramètre de pénalité
    breakpoints.append(result)

# Convertissez les indices des points de rupture en un seul ensemble
breakpoints = np.unique(np.concatenate(breakpoints))

# Affichez les points de rupture
print("Points de rupture détectés :", breakpoints)

# Réinitialiser l'index du dataframe
final_data = final_data.reset_index(drop=True)

# On écrit alors les points de rupture détectés
breakpoints = [0, 40, 85, 135, len(final_data) - 1]

Points de rupture détectés : [ 40  85 135 172]


Nous avons utilisé l'algorithme Pelt de la bibliothèque "ruptures" afin de détecter les points de rupture. Ce modèle Pelt utilise le modèle "rbf" (radial basis function) pour estimer le coût d'ajustement entre segments successifs de la série temporelle des données économiques. L'algorithme fonctionne de la manière suivante : on initie une segmentation triviale, puis on propage et ajuste itérativement cette segmentation tout en écartant les branches non prometteuses. La détection de ruptures repose sur la minimisation du coût global de la segmentation. L'Nous avons utilisé ce modèle car il permet d'identifier des périodes où les coefficients de la régression linéaire présentent des changements significatifs, facilitant ainsi l'analyse des influences temporelles sur la dette de l'État.

Ainsi, nous trouvons 4 périodes. La première démarre en janvier 2009 et se termine en mai 2012, la seconde démarre en mai 2012 et se termine en février 2016. La troisième commence en février 2016 et s'achève en avril 2020. La dernière commence en avril 2020 et se termine en 2023, lors de nos dernières données. 

Une fois ces périodes trouvées, on peut alors reproduire une régression linéaire pour chaque segment afin de visualiser, commenter et interpréter les coefficients de chaque segment. 

In [8]:
# Extraire les dates correspondantes aux points de rupture
breakpoint_dates = final_data.loc[breakpoints, 'DATE']

# Ajustez les dates pour le premier et le dernier segment
breakpoint_dates.iloc[0] = pd.to_datetime('2009-01-01')
breakpoint_dates.iloc[-1] = pd.to_datetime('2023-04-01')

# Diviser le dataframe en segments basés sur les dates de rupture
segments = []
for i in range(len(breakpoints) - 1):
    start_date = pd.to_datetime(breakpoint_dates.iloc[i])
    end_date = pd.to_datetime(breakpoint_dates.iloc[i + 1])
    segment_data = final_data[(final_data['DATE'] >= start_date) & (final_data['DATE'] < end_date)]
    segments.append(segment_data)

# Ajuster une régression linéaire robuste (Huber) pour chaque segment avec statsmodels
for i, segment_data in enumerate(segments):
    start_date = pd.to_datetime(breakpoint_dates.iloc[i])
    end_date = pd.to_datetime(breakpoint_dates.iloc[i + 1])

    X = segment_data.drop(columns=['negotiable_debts', 'DATE', 'Deposit facility', 'Marginal lending facility'])
    Y = segment_data['negotiable_debts']

    # Ajoutez une constante à X pour estimer l'ordonnée à l'origine
    X = sm.add_constant(X)

    # Créer un modèle de régression linéaire robuste (Huber) avec statsmodels    
    model = sm.RLM(Y, X, M=sm.robust.norms.HuberT()).fit()

    # Afficher les résultats sous forme de tableau
    print(f"\nSegment {i + 1} - Period: {start_date} to {end_date}")
    print(model.summary())


Segment 1 - Period: 2009-01-01 00:00:00 to 2012-05-01 00:00:00
                    Robust linear Model Regression Results                    
Dep. Variable:       negotiable_debts   No. Observations:                   40
Model:                            RLM   Df Residuals:                       31
Method:                          IRLS   Df Model:                            8
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Sat, 30 Dec 2023                                         
Time:                        21:21:58                                         
No. Iterations:                     2                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------

La première période (2009-avril 2012) est une période post-crise 2008. Comme attendu et commenté plus haut, le coefficient devant la colonne "Balance of payments", "unemployment_rate", "Valuation" sont significatifs et positif. Les coefficients devant "firms_creation", "natality_rate", "Exch_rate" et "Fixed rate" sont non significatifs sûrement dû au faible nombre de données sur cette période de 3 ans. Le coefficients devant 'business_insolvencies" est négatif mais assez faible, tout comme dans la régression globale, ce qui s'interprète donc de la même manière. Comme changement notable, la période 2009-2012 montre une amplification de l'effet de la balance des paiements sur la dette, et une relative stabilité des insolvabilités par rapport à la période complète

La seconde période, entre 2012 et 2016, est marquée par une stagnation économique, un chômage grimpant et une crise dans les échanges européens. 
Comparé au segment précédent, plusieurs changements dans les coefficients sont notables :
Le coefficient devant "BOP" diminue à 4.28, montrant une moindre dépendance aux échanges internationaux. Cela peut refléter une période de stabilisation après une croissance rapide des échanges dans le segment précédent.
Peu de coefficients sont en fait toujours significatif au seuil de 10%.

La troisième période, entre 2016 et 2020, est toujours une période dde stagnation économique. Le coeffiicient devant le taux de chômage devient significatif au niveau 1% et est trés bas. Le coefficient devant "Valuation" est significatif, comme à la première période, mais cette fois de signe opposé, il est désormais négatif. Un coefficient négatif devant "Valuation" suggère qu'une dépréciation de l'euro par rapport au dollar est associée à une diminution de la dette publique en France. Cela peut résulter de l'impact positif sur les exportations, réduisant la nécessité d'emprunter. De plus, si une part significative de la dette est libellée en devises étrangères, une dépréciation augmente sa valeur en euros. Cependant, l'interprétation dépend de divers facteurs économiques.

La dernière période, après avril 2020 est la période durant laquelle le COVID a fragilisé l'économie française. Les seuls régresseurs ayant des coefficients significatifs sont "BOP", "Fixed rate" et "Natality_rate". Ceci ne semble pas absurde dans une situation où les échanges commerciaux ont connu de fortes fluctuations (à la baisse puis à la hausse), et à un moment où la BCE a relevé ses taux afin de limiter l'inflation. Pourtant non significatif aux périodes précédentes, le coefficient devant "Natality_rate", fortement négatif, devient significatif au seuil de 1%. C'est donc uniquement sur cette période que ce coefficient est significatif, sachant qu'il l'est sur la régression faite sur l'ensemble des données. 

## Essai et comparaison de nouveaux modèles prédictifs

La dernière partie de ce projet sur la dette négociable de l'Etat français vise à étudier d'autres formes de régressions, la régression linéaire n'étant pas unique. Après plusieurs recherches sur leur existence et sur leur implantation informatique, nous avons trouvé les Arbres de décisions, les Forêts aléatoires et les modèles Adaboost. 
On commencer par évaluer  quatre modèles (Régression Linéaire, Arbre de Décision, Forêt Aléatoire et AdaBoost) en utilisant la validation croisée pour calculer les scores RMSE (Root Mean Squared Error) pour chaque pli et affiche la moyenne RMSE pour chaque modèle.

Le principe ici est le suivant : La validation croisée divise l'ensemble de données en plusieurs plis et évalue le modèle sur chacun d'entre eux.
On entraine alors le modèle plusieurs fois selon le nombre de division, chaque fois en utilisant un pli différent comme ensemble de test et les autres comme ensembles d'entraînements. Puis pour chaque pli on évalue la performance en calculant le RMSE, et on fait enfin la moyenne des RMSE. 



In [9]:
df = final_data[selected_columns]
df = df.dropna()
X = df.drop('negotiable_debts', axis=1)
y = df['negotiable_debts']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Les Modèles qu'on a choisit
models = {
    'Régression Linéaire': LinearRegression(),
    'Arbre de Décision': DecisionTreeRegressor(),
    'Forêt Aléatoire': RandomForestRegressor(),
    'AdaBoost': AdaBoostRegressor()
}

# Évaluation des modèles avec la validation croisée
for model_name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = pd.Series(-scores, name="RMSE").apply(lambda x: round(x ** 0.5, 2))
    print(f"Modèle: {model_name}")
    print("RMSE scores:", rmse_scores)
    print(f"Moyenne RMSE: {rmse_scores.mean()}\n")

Modèle: Régression Linéaire
RMSE scores: 0    59336.03
1    66908.69
2    68466.28
3    63349.78
4    52888.35
Name: RMSE, dtype: float64
Moyenne RMSE: 62189.826



Modèle: Arbre de Décision
RMSE scores: 0     40751.59
1    211971.48
2     82041.12
3    108085.41
4     31846.40
Name: RMSE, dtype: float64
Moyenne RMSE: 94939.2

Modèle: Forêt Aléatoire
RMSE scores: 0     23597.67
1    143179.64
2     69740.45
3     33073.32
4     70090.54
Name: RMSE, dtype: float64
Moyenne RMSE: 67936.324

Modèle: AdaBoost
RMSE scores: 0    25348.86
1    30737.27
2    56681.79
3    31361.72
4    29860.40
Name: RMSE, dtype: float64
Moyenne RMSE: 34798.008



### Interprétation des Résultats :

### Régression Linéaire :
Les RMSE varient significativement d'un pli à l'autre.
La moyenne RMSE est relativement élevée (62189.83), indiquant une dispersion importante des erreurs de prédiction.

###Arbre de Décision :
Les RMSE sont très variables, avec un écart élevé entre le meilleur et le pire pli.
La moyenne RMSE est élevée (80897.62), suggérant une performance mitigée du modèle.

### Forêt Aléatoire :
Les RMSE montrent une variabilité, mais moins prononcée que pour l'Arbre de Décision.
La moyenne RMSE (68686.13) est relativement élevée, indiquant que le modèle ne parvient pas à prédire de manière précise.

### AdaBoost :
Les RMSE sont plus homogènes entre les plis par rapport à d'autres modèles.
La moyenne RMSE (35294.03) est relativement basse, suggérant une meilleure performance en termes de précision de prédiction.

Le modèle avec la moyenne RMSE la plus faible est donc le modèle Adaboost. 

## Deuxième phase : prédiction et comparaison des modèles
L'approche ici est différente. L'objectif ici est d'appliquer les 4 modèles de régression afin de prédire la variable cible (la dette) à un instant T, par les données des régresseurs aux instants [2009-01-01 - T]. Puis, dans un second temps, comparer les prédictions des modèles entre eux, à chaque date, et les comparer à la vraie valeur de la dette à cette date donnée. 
Ensuite, le code utilise une boucle pour itérer sur les dates, entraîne les modèles sur l'ensemble d'entraînement à chaque itération, fait des prédictions sur l'ensemble de test, et stocke ces prédictions dans le DataFrame final_data. Enfin, il affiche la somme des MSE (Mean Squared Error) pour chaque modèle.

Le principe est alors le suivant : La boucle itère sur chaque date à partir de la deuxième date dans le jeu de données.À chaque itération, les données jusqu'à la date actuelle sont utilisées pour l'entraînement, tandis que les données à la date actuelle sont utilisées pour les tests.
On entraine ensuite le modèle : Pour chaque date, les modèles sélectionnés (Régression Linéaire, Arbre de Décision, Forêt Aléatoire, AdaBoost) sont entraînés sur les caractéristiques sélectionnées jusqu'à la date actuelle.Puis on fait des prédictions à la datte actuelle, la prédiction est stockée et ajoutée à la base de données pour chaque modèles. Enfin on calcule la MSE (mean squared error) pour chaque modèle

In [10]:
import warnings
from tqdm import tqdm

# Créer un dictionnaire pour stocker les métriques pour chaque modèle
metrics_scores = {model_name: {'MSE': []} for model_name in models.keys()}

# Création d'un nouveau df pour stocker les prédictions
predictions_df = pd.DataFrame()
predictions_df.dropna()

# Variables explicatives
features = ['BOP', 'business_insolvencies', 'firms_creation', 'natality_rate', 'unemployment_rate', 'Valuation', 'Exch_rate', 'Fixed rate', 'negotiable_debts']

# Création d'un dictionnaire pour stocker les MSE pour chaque modèle
mse_scores = {model_name: [] for model_name in models.keys()}

# Désactiver les avertissements pendant l'exécution de la boucle
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # Boucle sur les dates à partir de la deuxième date
    for i in tqdm(range(1, len(final_data))):
        # Séparer les données en ensemble d'entraînement et de test
        train_data = final_data.iloc[:i, :]
        test_data = final_data.iloc[i, :]

        # Sélectionner les caractéristiques et la cible
        X_train = train_data[features]
        y_train = train_data['negotiable_debts']
        X_test = test_data[features]

        # Boucle sur les modèles
        for model_name, model in models.items():
            # Entraîner le modèle
            model.fit(X_train, y_train)

            # Faire la prédiction sur les données de test
            y_pred = model.predict(X_test.values.reshape(1, -1))

            # Ajouter les prédictions au DataFrame final_data
            final_data.at[test_data.name, f'{model_name}_Prediction'] = y_pred[0]

            # Calculer les métriques et les ajouter aux listes correspondantes
            mse = mean_squared_error([test_data['negotiable_debts']], [y_pred])

            metrics_scores[model_name]['MSE'].append(mse)

# Afficher les métriques pour chaque modèle
for model_name, metrics_dict in metrics_scores.items():
    total_mse = np.sum(metrics_dict['MSE'])
    print(f'Somme des MSE pour {model_name}: {total_mse}')
   
# Afficher les 5 dernières lignes
print("5 dernières lignes de final_data :")
print(final_data.tail())

# Afficher les 5 premières lignes
print("\n5 premières lignes de final_data :")
print(final_data.head())

100%|██████████| 171/171 [00:46<00:00,  3.70it/s]

Somme des MSE pour Régression Linéaire: 519500253.57943726
Somme des MSE pour Arbre de Décision: 87644044510.0
Somme des MSE pour Forêt Aléatoire: 202128015272.33408
Somme des MSE pour AdaBoost: 240566010187.07565
5 dernières lignes de final_data :
          DATE  negotiable_debts       BOP  business_insolvencies  \
167 2022-12-01           2277811  104521.0            4003.000000   
168 2023-01-01           2297631  109080.0            4872.000000   
169 2023-02-01           2319193  109849.0            4872.000000   
170 2023-03-01           2328956  109785.0            4872.000000   
171 2023-04-01           2352050  109339.0            4488.666667   

     firms_creation  natality_rate  unemployment_rate  Valuation  Exch_rate  \
167         89608.0           10.0           6.933333    6473.76     1.0702   
168         89832.0            9.6           6.900000    7082.42     1.0862   
169         85940.0            9.8           6.933333    7267.93     1.0576   
170         99475.0 

### Somme des MSE pour Chaque Modèle :

La somme des MSE est la plus basse pour la Régression Linéaire, indiquant une performance globale relativement meilleure par rapport aux autres modèles.
L'Arbre de Décision a une somme des MSE très élevée, soulignant une faible capacité du modèle à généraliser.
Forêt Aléatoire et AdaBoost ont des sommes intermédiaires, avec AdaBoost présentant une performance légèrement meilleure.

### Pourquoi les résultats diffèrent selon l'approche par les RMSE et les MSE ? 
Le RMSE moyen privilégie les modèles qui minimisent les erreurs pour chaque pli, même si cela conduit à quelques erreurs importantes.
La somme des MSE évalue les modèles en fonction de la contribution totale des erreurs, ce qui peut montrer que la Régression Linéaire a une performance plus stable sur l'ensemble des donnée. Cela peut signifier que la Régression Linéaire a moins tendance à générer des erreurs très importantes. 

### Conclusion 
En conclusion, même si le RMSE moyen pour AdaBoost semble meilleur, la somme des MSE indique que la Régression Linéaire peut présenter une performance plus stable et globalement meilleure sur l'ensemble des données. 